# BE case study

This case study concerns the fluorescence measured from the BE mutant of a cyanobacterium which contains only photosystem I (PSI). The measurement and the global analysis are described in ([Tian et al. 2013][tian2013]). 
The target analysis is based upon the model explained in ([van Stokkum 2018][vanstokkum2018]).

[tian2013]: https://doi.org/10.1039/C3CP43813A "Tian L, Farooq S, van Amerongen H (2013) Probing the picosecond kinetics of the photosystem II core complex in vivo. Physical Chemistry Chemical Physics 15 (9):3146-3154."

[vanstokkum2018]: https://www.doi.org/10.1201/9781351242899-20 "Systems biophysics: Global and target analysis of light harvesting and photochemical quenching in vivo. In: Croce R, van Grondelle R, van Amerongen H, van Stokkum I (eds) Light Harvesting in Photosynthesis. CRC Press, Boca Raton, pp 467-482."

# Install pyglotaran from github
!pip install https://github.com/glotaran/pyglotaran.git

!pip install https://github.com/glotaran/pyglotaran-extras.git

### Inspect data

In [ ]:
from pyglotaran_extras import plot_data_overview

BE_data_path = "BE_700_ma_tr2_gain30_30uW_RT_AV_bc_sh_sel_620_830.ascii"
plot_data_overview(BE_data_path, nr_of_data_svd_vectors=3);

## Global Analysis

### Used model and parameters

In [ ]:
from glotaran.utils.ipython import display_file

BE_global_model_path = "models/BE-global-model.yml"
BE_global_parameters_path = "models/BE-global-parameters.yml"

#### Model file

In [ ]:
display_file(BE_global_model_path, syntax="yaml")

#### Parameters file

In [ ]:
display_file(BE_global_parameters_path, syntax="yaml")

The free parameters are the three kinetic parameters and the IRF parameters. The IRF consists of a sum of two Gaussians with a common center. The IRF parameters are the center (the location of the maximum of the IRF) and the widths of the two Gaussians, and the relative amplitude of the second Gaussian.
Note that two dispersion parameters (to describe the wavelength dependence of the IRF center with a parabola) have been used, which have been estimated from the data as well.

### Create scheme and optimize it

In [ ]:
from glotaran.analysis.optimize import optimize
from glotaran.project.scheme import Scheme

BE_global_scheme = Scheme(
    model=BE_global_model_path,
    parameters=BE_global_parameters_path,
    data={"BE_data": BE_data_path},
    maximum_number_function_evaluations=11,
)
BE_global_scheme.validate()

In [ ]:
compartments = BE_global_scheme.model.initial_concentration["input1"].compartments

BE_global_scheme.model.k_matrix["km1"].matrix_as_markdown(compartments).replace("0.0000e+00", "")

In [ ]:
BE_global_result = optimize(BE_global_scheme)

In [ ]:
BE_global_result

In [ ]:
BE_global_result.optimized_parameters

### Result plots

#### Fit quality

In [ ]:
from pyglotaran_extras import plot_fitted_traces, select_plot_wavelengths

wavelengths = select_plot_wavelengths(
    BE_global_result.data["BE_data"], wavelength_range=(670, 750)
)
plot_fitted_traces(BE_global_result.data["BE_data"], wavelengths, linlog=True, linthresh=50);

#### Overview

In [ ]:
from pyglotaran_extras import plot_overview

plot_overview(
    BE_global_result.data["BE_data"],
    linlog=False,
    figure_only=False,
    nr_of_data_svd_vectors=3,
    nr_of_residual_svd_vectors=1,
);

In [ ]:
from pyglotaran_extras import plot_overview

plot_overview(
    BE_global_result.data["BE_data"],
    figure_only=False,
    nr_of_data_svd_vectors=3,
    nr_of_residual_svd_vectors=1,
    linlog=True,
    linthresh=50,
);

## Target Analysis

### Used model and parameters

In [ ]:
BE_target_model_path = "models/BE-target-model.yml"
BE_target_parameters_path = "models/BE-target-parameters.yml"

#### Model file

In [ ]:
display_file(BE_target_model_path, syntax="yaml")

#### Parameters file

In [ ]:
display_file(BE_target_parameters_path, syntax="yaml")

The free parameters are the free kinetic parameters and the free IRF parameters. Note that equal_area_penalties are needed to estimate the relative input to the freeChla (s4) and the equilibrium of PS1bulk and PS1red (s1 and s2) and that the radical pair RP1 (s3) has zero emission. The natural decay rate of Chla has been fixed at 1/(2 ns).

### Create scheme and optimize it

In [ ]:
BE_target_scheme = Scheme(
    model=BE_target_model_path,
    parameters=BE_target_parameters_path,
    data={"BE_data": BE_data_path},
    maximum_number_function_evaluations=13,
)
BE_target_scheme.validate()

In [ ]:
compartments = BE_target_scheme.model.initial_concentration["input1"].compartments

BE_target_scheme.model.k_matrix["PS1"].matrix_as_markdown(compartments).replace("0.0000e+00", "")

In [ ]:
BE_target_result = optimize(BE_target_scheme)

In [ ]:
BE_target_result

### Result plots

#### Fit quality

In [ ]:
plot_fitted_traces(BE_target_result.data["BE_data"], wavelengths, linlog=True, linthresh=50);

#### Overview

In [ ]:
plot_overview(
    BE_target_result.data["BE_data"],
    figure_only=False,
    nr_of_data_svd_vectors=3,
    nr_of_residual_svd_vectors=1,
    linlog=True,
    linthresh=50,
);

In [ ]:
BE_target_result.optimized_parameters

In [ ]:
BE_target_result.data["BE_data"]